# Investment model dispatch patterns
Routines for analyzing dispatch patterns of *pommesinvest* runs

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, aggregate_investment_results, plot_single_dispatch_pattern
)

## Parameters and workflow settings

In [ ]:
# Model configuration
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "50"
dr_scenarios = ["none", "5", "50", "95"]
fuel_price_scenario = "NZE_high"
emissions_pathway = "long-term"
european_data = True
countries = [
    "AT",
    "BE",
    "CH",
    "CZ",
    "DE",
    "DK1",
    "DK2",
    "FR",
    "NL",
    "NO1",
    "NO2",
    "NO3",
    "NO4",
    "NO5",
    "PL",
    "SE1",
    "SE2",
    "SE3",
    "SE4",
    "IT",
]

# Paths, filenames and color codes
path_results = "./model_results/pommesinvest/"
path_processed_data = "./data_out/"
path_plots = "./plots/"

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if dr_scenario != "none":
    file_add_on = (
        f"_with_dr_{dr_scenario}_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}_production"
    )
else:
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}_production"
    )
file_extension = ".csv"

filenames_out = {
    "shortages_artificial": {
        dr_scenario: f"{path_processed_data}sources_el_artificial_{dr_scenario}.csv"
        for dr_scenario in dr_scenarios
    },
    "shortages_artificial_ts": {
        dr_scenario: f"{path_processed_data}sources_el_artificial_ts_{dr_scenario}.csv"
        for dr_scenario in dr_scenarios
    }
}

# color codes
FUELS_EXISTING = {
    "uranium": "#e50000",
    "lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "mixedfuels": "#a57e52",
    "otherfossil": "#d8dcd6",
}

FUELS = {
    "biomass": "#15b01a",
    "hydrogen": "#6fa8dc",
    "natgas": "#ffd966",
    "oil": "#aaa662",
    "waste": "#c04e01"
}

RES_SOURCES = {
    "DE_source_ROR": "#c79fef",
    "DE_source_biomassEEG": "#15b01a",
    "DE_source_geothermal": "#cccccc",
    "DE_source_landfillgas": "#cccccc",
    "DE_source_larga": "#cccccc",
    "DE_source_minegas": "#cccccc",
    "DE_source_solarPV": "#fcb001",
    "DE_source_windoffshore": "#0504aa",
    "DE_source_windonshore": "#82cafc",
}

DEMAND_RESPONSE_CLUSTERS = {
    "hoho_cluster_shift_only": "#333333", 
    "hoho_cluster_shift_shed": "#555555", 
    "ind_cluster_shed_only": "#666666",
    "ind_cluster_shift_only": "#888888", 
    "ind_cluster_shift_shed": "#aaaaaa", 
    "tcs+hoho_cluster_shift_only": "pink", # "#cccccc",
    "tcs_cluster_shift_only": "orange" # "#dddddd", 
}

STORAGES = {
    "PHS": "#0c2aac",
    "PHS_new_built": "#7c90e7",
    "battery": "#f7e09a",
    "battery_new_built": "#fff5d5",
}

LOAD = {
    "DE_sink_el_load": "darkblue"
}

SHORTAGE_EXCESS = {
    "DE_sink_el_excess": "purple",
    "DE_source_el_shortage": "red",
}

# Workflow and output configuration
plt.rcParams.update({'font.size': 12})
rounding_precision = 2

start_time_step = "2037-03-03 00:00:00"
time_steps_to_be_considered_in_hours = 168 * 12
amount_of_time_steps = time_steps_to_be_considered_in_hours / int(frequency.split("H")[0])

# Single scenario analyses
Inspect the results for a single scenario model run.
## Read in, preprocess and aggregate data
* Use routine originally developped for investment model to preprocess raw results. Therefore, transpose back and forth.
* Aggregate by fuel. Don't aggregate storages, demand response etc.
* Form distinct data sets:
    * Demand: regular load without demand response baseline consumption and demand response net load
    * Exports and imports: exports from DE to European neighbours, imports vice versa
    * Storages: inflow, outflow and net storage usage derived from these
    * Generators: generation aggregated per fuel
    * Demand Response: upshifts, downshifts and demand response storage level
    * Shortage and excess

In [ ]:
if european_data:
    header = [0, 1]
else:
    header = 0
production_results_raw = pd.read_csv(
    f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0, header=header
).T
processed_results = preprocess_raw_results(
    production_results_raw, investments=False, european_data=european_data
).drop(columns="year").round(rounding_precision)
aggregated_results = aggregate_investment_results(
    processed_results, energy_carriers={**FUELS_EXISTING, **FUELS}, by="energy_carrier", investments=False
).T
del production_results_raw, processed_results

In [ ]:
# Define cols to group
demand_cols = [col for col in aggregated_results.columns if "DE_sink_el" in col and "_excess" not in col]
export_link_cols = [col for col in aggregated_results.columns if "DE_link" in col]
import_link_cols = [col for col in aggregated_results.columns if "link_DE" in col]
power_prices_col = ["DE_bus_el"]

all_demand_response_cols = list(set([
    col for col in aggregated_results.columns for key in DEMAND_RESPONSE_CLUSTERS if key in col
]))

demand_response_after_cols = [col for col in all_demand_response_cols if "_demand_after" in col]
demand_cols.extend(demand_response_after_cols)

demand_response_other_cols = [
    col for col in all_demand_response_cols 
    if col not in demand_response_after_cols
    # Exclude fictious demand response storage level which can be calculated ex post
    and not "storage_level" in col
]

storages_cols = list(set([
    col for col in aggregated_results.columns 
    for key in STORAGES if key in col
    and "inflow" in col or "outflow" in col
]))
shortage_excess_cols = [
    col for col in aggregated_results.columns if col in ["DE_sink_el_excess", "DE_source_el_shortage"]
]
electrolyzer_cols = [col for col in aggregated_results.columns if "electrolyzer" in col]
foreign_country_cols = list(
    set([
        col for col in aggregated_results.columns for country in countries
        if f"{country}_" in col and country != "DE"
    ])
)

generators_cols = [
    col for col in aggregated_results.columns 
    if col not in demand_cols 
    and col not in export_link_cols
    and col not in import_link_cols
    and col not in all_demand_response_cols 
    and col not in storages_cols
    and col not in shortage_excess_cols
    and col not in electrolyzer_cols
    and col not in power_prices_col
    and col not in foreign_country_cols
]

# Split overall data set to distinct subsets
demand_pattern = aggregated_results[demand_cols]
export_pattern = aggregated_results[export_link_cols]
import_pattern = aggregated_results[import_link_cols]
demand_response_pattern = aggregated_results[demand_response_other_cols]
storages_pattern = aggregated_results[storages_cols]
shortage_excess_pattern = aggregated_results[shortage_excess_cols]
generators_pattern = aggregated_results[generators_cols]
power_prices_pattern = aggregated_results[power_prices_col]

# Slightly alter / negate
storages_pattern.loc[:, [col for col in storages_pattern.columns if "_outflow" in col]] *= (-1)
shortage_excess_pattern.loc[:, "DE_source_el_shortage"] *= (-1)
demand_response_pattern.loc[: , [col for col in demand_response_pattern.columns if "dsm_do" in col]] *= (-1)

In [ ]:
generators_pattern.sum()

## Inspect some long-term results
* Exports and imports patterns
* Demand

### Exports and imports

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
export_pattern.plot(kind="area", stacked=True, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
import_pattern.plot(kind="area", stacked=True, ax=ax)
plt.show()

In [ ]:
if dr_scenario != "none":
    fig, ax = plt.subplots(len(demand_pattern.columns), figsize=(16, 8 * len(demand_pattern.columns)))
    for i, col in enumerate(demand_pattern.columns):
        demand_pattern[col].plot(kind="area", stacked=True, ax=ax[i])
        ax[i].legend(loc="best")
    plt.show()

else:
    fig, ax = plt.subplots(figsize=(16, 8))
    demand_pattern.plot(kind="area", ax=ax)
    plt.show()                           

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
demand_pattern.plot(kind="area", stacked=True, ax=ax)
plt.show()

In [ ]:
demand_pattern.max()

## Create simple dispatch plots
### Demand and generation
Create simple area plots

In [ ]:
colors = {
    **LOAD, 
    **{
        f"{cluster}_demand_after": value for cluster, value in DEMAND_RESPONSE_CLUSTERS.items()
    }
}

plot_single_dispatch_pattern(
    demand_pattern,
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="demand_pattern",
)

In [ ]:
demand_pattern.describe()

In [ ]:
generators_pattern.describe()

In [ ]:
generators_pattern["mixedfuels"].max()

In [ ]:
start_time_step="2037-03-03 16:00:00"
amount_of_time_steps=168 * 2

In [ ]:
colors = {
    **LOAD, 
    **{
        f"{cluster}_demand_after": value for cluster, value in DEMAND_RESPONSE_CLUSTERS.items()
    }
}

plot_single_dispatch_pattern(
    demand_pattern,
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="demand_pattern",
)

In [ ]:
colors = {
    **FUELS_EXISTING, 
    **FUELS,
    **RES_SOURCES,
}

plot_single_dispatch_pattern(
    generators_pattern,
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="generation_pattern",
)

### Storages, Demand Response, Shortage and excess
Area plots, but change in sign

In [ ]:
shortage_excess_pattern.describe()

In [ ]:
colors = SHORTAGE_EXCESS

plot_single_dispatch_pattern(
    shortage_excess_pattern,
    "2020-01-01 00:00:00",
    227758,
    colors,
    save=True,
    path_plots="./plots/",
    filename="shortage_excess_pattern",
    kind="line"
)

In [ ]:
storages_pattern.describe()

In [ ]:
colors = {
    **{f"{storage}_outflow": color for storage, color in STORAGES.items()},
    **{f"{storage}_inflow": color for storage, color in STORAGES.items()}
}

plot_single_dispatch_pattern(
    storages_pattern,
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="storages_pattern",
)

In [ ]:
demand_response_pattern.describe()

In [ ]:
colors = {
    **{f"{cluster}_dsm_up": color for cluster, color in DEMAND_RESPONSE_CLUSTERS.items()},
    **{f"{cluster}_dsm_do_shift": color for cluster, color in DEMAND_RESPONSE_CLUSTERS.items()}
}

plot_single_dispatch_pattern(
    demand_response_pattern[[col for col in demand_response_pattern.columns if not "dsm_do_shed" in col]],
    start_time_step,
    amount_of_time_steps,
    colors,
    save=True,
    path_plots="./plots/",
    filename="demand_response_pattern",
)

In [ ]:
demand_response_pattern.loc["2037-03-08 00:00":"2037-03-09 23:00", 
                            ["ind_cluster_shift_only_dsm_do_shift", "ind_cluster_shift_only_dsm_up"]]

# Multi scenario analyses
## Extract dispatch results

In [ ]:
aggregated_results = {}

for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        file_add_on = (
            f"_with_dr_{dr_scenario}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}_production"
        )
    else:
        file_add_on = (
            f"_no_dr_50_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}_production"
        )
    if european_data:
        header = [0, 1]
    else:
        header = 0
    
    production_results_raw = pd.read_csv(
        f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0, header=header,
    ).T
    processed_results = preprocess_raw_results(
        production_results_raw, investments=False, european_data=european_data
    ).drop(columns="year").round(rounding_precision)
    aggregated_results[dr_scenario] = aggregate_investment_results(
        processed_results, 
        energy_carriers={**FUELS_EXISTING, **FUELS}, 
        by="energy_carrier", 
        investments=False
    ).T
del production_results_raw, processed_results

## Shortage events for other countries
* Inspect occurences of shortage for countries other than Germany.
* Prepare as a model input:
    * Artificial shortage sources with a fixed profile are added to pommesdata to prevent shortages.
    * Realized shortage profiles are extracted and used as fixed profile for pommesdata.

Extraction and inspection:
* Extract shortage patterns
* Visualize shortage time series

In [ ]:
shortages = {}
fig, axs = plt.subplots(len(dr_scenarios), 1, figsize=(15, 5 * len(dr_scenarios)))

for no, dr_scenario in enumerate(dr_scenarios):
    shortage_cols = [
        col for col in aggregated_results[dr_scenario].columns if "shortage" in col
    ]
    shortages[dr_scenario] = aggregated_results[dr_scenario][shortage_cols]
    _ = shortages[dr_scenario].plot(ax=axs[no])
    _ = axs[no].set_title(f"shortage events for scenario {dr_scenario}")
    
plt.show()

## Maximum shortage capacities
* Extract maximum storage capacities
* add oemof.solph-relevant pieces of information and save to file

In [ ]:
for dr_scenario in dr_scenarios:
    shortages_max = pd.DataFrame(index=shortages[dr_scenario].columns)
    shortages_max.index.name = "label"
    shortages_max["to"] = shortages_max.index.str.split("_", expand=True).get_level_values(0) + "_bus_el"
    shortages_max["maximum"] = shortages[dr_scenario].max()
    shortages_max.index = shortages_max.index + "_artificial"
    shortages_max = shortages_max.loc[shortages_max.maximum != 0]
    shortages_max.to_csv(filenames_out["shortages_artificial"][dr_scenario])

## Shortage profiles
* Extract normalized shortage profiles
* Store time series to file

In [ ]:
for dr_scenario in dr_scenarios:
    shortages_profile = shortages[dr_scenario].div(shortages[dr_scenario].max())
    shortages_profile.columns = [col + "_artificial" for col in shortages_profile.columns]
    shortages_profile = shortages_profile[[
        col for col in shortages_profile.columns
        if shortages_profile[col].notna().all()
    ]]
    shortages_profile.to_csv(filenames_out["shortages_artificial_ts"][dr_scenario])